# Configuration Migration Example

This notebook demonstrates how to migrate from the old configuration system to the new ConfigManager.

## Old Way (Deprecated)
The old system used direct imports and manual configuration creation:

In [ ]:
# OLD WAY - Direct configuration import (deprecated)
import sys
from pathlib import Path

# Add parent src directory to path for imports
sys.path.insert(0, str(Path().absolute().parent / 'src'))

from config import ManufacturerConfig
from manufacturer import WidgetManufacturer

# Manual configuration
config = ManufacturerConfig(
    initial_assets=10_000_000,
    asset_turnover_ratio=1.0,
    base_operating_margin=0.08,
    tax_rate=0.25,
    retention_ratio=1.0
)

manufacturer = WidgetManufacturer(config)
print(f"Old way - Assets: ${manufacturer.assets:,.0f}")

Old way - Assets: $10,000,000


## New Way (Recommended)
The new system uses ConfigManager with profiles, modules, and presets:

In [2]:
# NEW WAY - Using ConfigManager (recommended)
import sys
from pathlib import Path

# Add parent src directory to path for imports
sys.path.insert(0, str(Path().absolute().parent / 'src'))

from config_manager import ConfigManager
from manufacturer import WidgetManufacturer

# Load configuration from profile
manager = ConfigManager()
config = manager.load_profile("default")

# Access manufacturer config
manufacturer = WidgetManufacturer(config.manufacturer)
print(f"New way - Assets: ${manufacturer.assets:,.0f}")
print(f"Profile: {config.profile.name}")
print(f"Description: {config.profile.description}")

New way - Assets: $10,000,000
Profile: default
Description: Standard baseline configuration for widget manufacturer


## Benefits of the New System

### 1. Profile Variants
Easily switch between predefined configurations:

In [ ]:
# Load different profiles
profiles = ["default", "conservative", "aggressive"]

for profile_name in profiles:
    config = manager.load_profile(profile_name)
    print(f"\n{profile_name.upper()} Profile:")
    print(f"  Base operating margin: {config.manufacturer.base_operating_margin:.1%}")
    print(f"  Tax rate: {config.manufacturer.tax_rate:.1%}")
    print(f"  Growth rate: {config.growth.annual_growth_rate:.1%}")


DEFAULT Profile:
  Operating margin: 10.0%
  Tax rate: 25.0%
  Growth rate: 12.0%

CONSERVATIVE Profile:
  Operating margin: 6.0%
  Tax rate: 28.0%
  Growth rate: 3.0%

AGGRESSIVE Profile:
  Operating margin: 12.0%
  Tax rate: 21.0%
  Growth rate: 8.0%


### 2. Runtime Overrides
Override specific parameters without changing files:

In [ ]:
# Override specific parameters using double underscore notation
config = manager.load_profile(
    "default",
    manufacturer__operating_margin=0.12,
    growth__annual_growth_rate=0.15
)

print("Configuration with overrides:")
print(f"  Base operating margin: {config.manufacturer.base_operating_margin:.1%} (overridden)")
print(f"  Growth rate: {config.growth.annual_growth_rate:.1%} (overridden)")
print(f"  Tax rate: {config.manufacturer.tax_rate:.1%} (default)")

Configuration with overrides:
  Operating margin: 12.0% (overridden)
  Growth rate: 15.0% (overridden)
  Tax rate: 25.0% (default)


### 3. Presets for Common Scenarios
Apply predefined market conditions or risk scenarios:

In [5]:
# Apply market condition presets
config_stable = manager.load_profile("default", presets=["stable"])
config_volatile = manager.load_profile("default", presets=["volatile"])

print("Market Condition Presets:")
# Note: The presets attempt to set revenue_volatility on ManufacturerConfig, 
# but this attribute doesn't exist in the current schema.
# Instead, volatility is part of the GrowthConfig
print(f"  Default growth volatility: {config_stable.growth.volatility}")
print(f"  Growth type: {config_stable.growth.type}")

# Apply growth preset which correctly modifies the growth config
config_growth = manager.load_profile("default", presets=["growth"])
config_recession = manager.load_profile("default", presets=["recession"])

print("\nGrowth Presets:")
print(f"  Growth scenario - Annual rate: {config_growth.growth.annual_growth_rate:.1%}")
print(f"  Recession scenario - Annual rate: {config_recession.growth.annual_growth_rate:.1%}")

# Apply multiple presets
config = manager.load_profile(
    "default",
    presets=["growth"]
)
print("\nWith growth preset applied:")
print(f"  Annual growth rate: {config.growth.annual_growth_rate:.1%}")
print(f"  Growth type: {config.growth.type}")

Market Condition Presets:
  Default growth volatility: 0.0
  Growth type: deterministic

Growth Presets:
  Growth scenario - Annual rate: 12.0%
  Recession scenario - Annual rate: 12.0%

With growth preset applied:
  Annual growth rate: 12.0%
  Growth type: deterministic


### 4. Module Composition
Load only the modules you need:

In [6]:
# Load specific modules only
config = manager.load_profile(
    "default",
    modules=["insurance", "stochastic"]
)

print("Loaded modules:")
print(f"  Insurance: {hasattr(config, 'insurance') and config.insurance is not None}")
print(f"  Stochastic: {hasattr(config, 'stochastic') and config.stochastic is not None}")
print(f"  Losses: {hasattr(config, 'losses') and config.losses is not None}")

Loaded modules:
  Insurance: False
  Stochastic: False
  Losses: False


## Migration Checklist

When updating your notebooks:

1. ✅ Replace `from src.config import ManufacturerConfig` with `from src.config_manager import ConfigManager`
2. ✅ Load configurations using `manager.load_profile()` instead of manual creation
3. ✅ Access sub-configurations through the loaded config (e.g., `config.manufacturer`)
4. ✅ Use profile names: `"default"` instead of `"baseline"`, `"aggressive"` instead of `"optimistic"`
5. ✅ Leverage presets and overrides instead of modifying YAML files

## Backward Compatibility

The old ConfigLoader still works but shows deprecation warnings:

In [ ]:
# This still works but is deprecated
from config_loader import ConfigLoader

loader = ConfigLoader()
config = loader.load("baseline")  # Shows deprecation warning

print("ConfigLoader still works for backward compatibility")
print(f"Manufacturer base margin: {config.manufacturer.base_operating_margin:.1%}")

ConfigLoader still works for backward compatibility
Manufacturer margin: 10.0%


C:\Users\alexf\AppData\Local\Temp\ipykernel_34568\912590151.py:5: DeprecationWarning: ConfigLoader is deprecated. Please migrate to ConfigManager for improved functionality.
  config = loader.load("baseline")  # Shows deprecation warning


## Summary

The new ConfigManager provides:
- **Cleaner code**: Load complete configurations with one line
- **Flexibility**: Override parameters at runtime
- **Reusability**: Use profiles and presets across notebooks
- **Performance**: Built-in caching for faster loads
- **Maintainability**: Centralized configuration management

See `migration_guide.md` for complete documentation.